In [0]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import sqlite3
import csv
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from wordcloud import WordCloud
import re
import os
from sqlalchemy import create_engine # database connection
import datetime as dt
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.snowball import SnowballStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import SGDClassifier
from sklearn import metrics
from sklearn.metrics import f1_score,precision_score,recall_score
from sklearn import svm
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from datetime import datetime

# loading data

In [2]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

     |████████████████████████████████| 993kB 6.6MB/s 


In [3]:

link = 'https://drive.google.com/open?id=1TUDwODYHxNneCjyss6XhqJaDpB0NrQbG' # The shareable link
fluff, id = link.split('=')
print (id) 

1TUDwODYHxNneCjyss6XhqJaDpB0NrQbG


In [0]:
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('mpst_full_data.csv')  
 


In [5]:
df=pd.read_csv('mpst_full_data.csv')
df.head()

,imdb_id,title,plot_synopsis,tags,split,synopsis_source
0,tt0057603,I tre volti della paura,Note: this synopsis is for the orginal Italian...,"cult, horror, gothic, murder, atmospheric",train,imdb
1,tt1733125,Dungeons & Dragons: The Book of Vile Darkness,"Two thousand years ago, Nhagruul the Foul, a s...",violence,train,imdb
2,tt0033045,The Shop Around the Corner,"Matuschek's, a gift store in Budapest, is the ...",romantic,test,imdb
3,tt0113862,Mr. Holland's Opus,"Glenn Holland, not a morning person by anyone'...","inspiring, romantic, stupid, feel-good",train,imdb
4,tt0086250,Scarface,"In May 1980, a Cuban man named Tony Montana (A...","cruelty, murder, dramatic, cult, violence, atm...",val,imdb


In [6]:
df.shape

(14828, 6)


# preprocessing data

In [0]:
sort=df.sort_values('imdb_id')

In [8]:
f=sort.drop_duplicates(subset={'title','tags','plot_synopsis'})
f.shape

(14752, 6)

In [9]:
final=f.sort_values('split')
final.head()

,imdb_id,title,plot_synopsis,tags,split,synopsis_source
10426,tt6583664,Scapegoat,There are significant differences between this...,"revenge, suspenseful",test,wikipedia
10878,tt0076911,"What a Nightmare, Charlie Brown!","One winter day, Charlie Brown is trying to pre...",psychedelic,test,wikipedia
1776,tt0499448,The Chronicles of Narnia: Prince Caspian,"On a cloudless night in Narnia, under an eclip...","good versus evil, violence, fantasy, boring, r...",test,imdb
335,tt0077235,Big Wednesday,"Malibu, California, 1962. Matt the Enforcer (J...","cult, philosophical",test,imdb
5293,tt0498399,We Own the Night,"New York, November 1988: A new breed of narcot...","revenge, suspenseful, murder, violence, flashback",test,imdb


In [10]:
final['split'].value_counts()

train    9436
test     2957
val      2359
Name: split, dtype: int64

In [11]:

final[0:2957].tail()

,imdb_id,title,plot_synopsis,tags,split,synopsis_source
9499,tt0036707,Cheyenne Wildcat,"The president of Blue Springs Bank, Jason Hopk...",murder,test,wikipedia
3941,tt1427298,The Human Race,"An exciting, unpredictable and emotionally dra...",violence,test,imdb
4092,tt0052901,Horrors of the Black Museum,HORRORS OF THE BLACK MUSEUM In London a delive...,murder,test,imdb
12562,tt1548635,Siren,"One week before his wedding day, Jonah and his...",murder,test,wikipedia
10903,tt0050407,Forty Guns,"In the 1880s, Griff Bonnell (Barry Sullivan) a...",murder,test,wikipedia


In [12]:
final[2957:12393].tail()

,imdb_id,title,plot_synopsis,tags,split,synopsis_source
258,tt0096320,Twins,Julius Benedict and Vincent Benedict are frate...,"comedy, entertaining",train,wikipedia
10615,tt0095122,The Expendables,"The Expendables—leader Barney Ross, knife spec...",cult,train,wikipedia
6280,tt0096018,Running on Empty,Mike (Terry Serio) is a young man who is a bud...,"anti war, romantic, avant garde, dramatic",train,wikipedia
312,tt0096734,The 'Burbs,The film starts on a small cul-de-sac suburban...,"comedy, mystery, gothic, intrigue, insanity, p...",train,imdb
11311,tt0095626,Messenger of Death,Children play outside a rural Colorado home. T...,"good versus evil, revenge, suspenseful, murder...",train,wikipedia


In [0]:
stopwords= set(['br', 'the', 'i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've",\
            "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', \
            'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their',\
            'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', \
            'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', \
            'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', \
            'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after',\
            'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further',\
            'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more',\
            'most', 'other', 'some', 'such', 'only', 'own', 'same', 'so', 'than', 'too', 'very', \
            's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', \
            've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn',\
            "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn',\
            "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", \
            'won', "won't", 'wouldn', "wouldn't"])

In [14]:
import nltk
nltk.download('punkt')
from tqdm import tqdm
from bs4 import BeautifulSoup
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english")
preprocess_plot=[]
for s in tqdm(final['plot_synopsis'].values):
    s=re.sub(r'S*\d\S*','',s)
    s=re.sub('[^A-Za-z]+',' ',s)
    soup=BeautifulSoup(s)
    s=soup.get_text()
    s=re.sub(r'http\S+','',s)
    words=word_tokenize(s.lower())

    #Removing all single letter and and stopwords from question exceptt for the letter 'c'
    s=' '.join(str(stemmer.stem(j)) for j in words if j not in stopwords and (len(j)!=1 ))
    preprocess_plot.append(s.strip())

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


100%|██████████| 14752/14752 [02:50<00:00, 86.50it/s]


In [15]:

preprocess_title=[]
for s in tqdm(final['title'].values):
    s=re.sub(r'S*\d\S*','',s)
    s=re.sub('[^A-Za-z]+',' ',s)
    soup=BeautifulSoup(s)
    s=soup.get_text()
    s=re.sub(r'http\S+','',s)
    words=word_tokenize(s.lower())

    #Removing all single letter and and stopwords from question exceptt for the letter 'c'
    s=' '.join(str(stemmer.stem(j)) for j in words if j not in stopwords and (len(j)!=1 ))
    preprocess_title.append(s.strip())

100%|██████████| 14752/14752 [00:07<00:00, 2097.70it/s]


In [16]:
preprocess_tags=[]
for s in tqdm(final['tags'].values):
    s=re.sub(r' ','',s)
    s=re.sub(r',',' ',s)
    s=' '.join(e.lower() for e in s.split() )
    preprocess_tags.append(s.strip())


100%|██████████| 14752/14752 [00:00<00:00, 199448.06it/s]


In [17]:
preprocess_plot[1]

'one winter day charli brown tri pretend musher snoopi dog idea get charli brown pull fun ride sled night come comfort indoor charli brown indign snoopi adjust well home life remind snoopi fact arctic dog fed day meal larg consist cold meat raw fish snoopi blanch give look bad come conclus snoopi over civil under dogifi dog make scrumptious dinner five pizza larg milkshak eat snoopi goe bed doghous prompt wake find sled dog iditarod trail sled dog race alaska presum klondik gold rush serum run nome snoopi cruelli mistreat owner seen shadow silhouett speak much deeper version classic peanut adult waa waa waa languag fellow dog run rag deni food water dog take turn bark loud snoopi order let know inde outsid one scene break snow scene sled master stop honki tonk hungri snoopi sneak insid snatch sandwich mug root beer sit near piano feign play washington post march snoopi tri hand game poker keep poker face laugh loud reveal improb win hand five ace caus brawl leav snoopi escap next room 

In [18]:
preprocess_tags[2]

'goodversusevil violence fantasy boring romantic'

In [19]:
preprocess_tags[5]

'boring murder dramatic violence flashback claustrophobic sentimental'

# choosing correct tags and vectorizing them

In [0]:
vectorizer = CountVectorizer(tokenizer = lambda x: x.split(), binary='true')
multilabel_y = vectorizer.fit_transform(preprocess_tags)

In [21]:
multilabel_y.shape

(14752, 71)

# train cv and test split

In [0]:
x_train1=preprocess_plot[2957:12393]
x_test1=preprocess_plot[:2957]
x_cv1=preprocess_plot[12393:]

x_train2=preprocess_title[2957:12393]
x_test2=preprocess_title[:2957]
x_cv2=preprocess_title[12393:]

y_train = multilabel_y[2957:12393]
y_test = multilabel_y[:2957]
y_cv=   multilabel_y[12393:]

# 200 dim word2vec of plot

In [0]:
from gensim.models import Word2Vec
from sklearn.feature_extraction.text import TfidfVectorizer
trlst=[]
for s in x_train1:
    trlst.append(s.split())

In [0]:
w2vtr=Word2Vec(trlst,size=200,min_count=5, workers=4)

In [0]:
w2v_wordstr = list(w2vtr.wv.vocab)

In [0]:
model= TfidfVectorizer(min_df=10)
tfidf=model.fit_transform(x_train1)
dictonary=dict(zip(model.get_feature_names(),model.idf_))


In [0]:
tr=[]
x= model.get_feature_names()
for s in tqdm(x_train1):
    a=s.split()
    vec= np.zeros(200)
    sum=0
    for b in a:
        if b in w2v_wordstr and b in x:
            tfidf=dictonary[b]*(a.count(b)/len(b))
            vec+=w2vtr.wv[b]*tfidf
            sum+=tfidf
    if sum != 0:
        vec=vec/sum
        
    tr.append(vec)        
        
        

100%|██████████| 9436/9436 [23:37<00:00,  6.66it/s]


In [0]:
cv=[]
for s in tqdm(x_cv1):
    a=s.split()
    vec= np.zeros(200)
    sum=0
    for b in a:
        if b in w2v_wordstr and b in x:
            tfidf=dictonary[b]*(a.count(b)/len(b))
            vec+=w2vtr.wv[b]*tfidf
            sum+=tfidf
    if sum != 0:
        vec=vec/sum
        
    cv.append(vec)        
        

100%|██████████| 2359/2359 [06:11<00:00,  7.60it/s]


In [0]:
test=[]
for s in tqdm(x_test1):
    a=s.split()
    vec= np.zeros(200)
    sum=0
    for b in a:
        if b in w2v_wordstr and b in x:
            tfidf=dictonary[b]*(a.count(b)/len(b))
            vec+=w2vtr.wv[b]*tfidf
            sum+=tfidf
    if sum != 0:
        vec=vec/sum
        
    test.append(vec)        
        

100%|██████████| 2957/2957 [07:31<00:00,  6.51it/s]


# tfidf

In [0]:
vectorizer =TfidfVectorizer(min_df=10,ngram_range=(1,3),max_features=20000)
x_train_multilabel1 = vectorizer.fit_transform(x_train1)
x_cv_multilabel1 = vectorizer.transform(x_cv1)
x_test_multilabel1 = vectorizer.transform(x_test1)

In [0]:
print("Dimensions of train data X:",x_train_multilabel1.shape, "Y :",y_train.shape)
print("Dimensions of cv data X:",x_cv_multilabel1.shape, "Y :",y_cv.shape)
print("Dimensions of test data X:",x_test_multilabel1.shape,"Y:",y_test.shape)

Dimensions of train data X: (9436, 20000) Y : (9436, 71)
Dimensions of cv data X: (2359, 20000) Y : (2359, 71)
Dimensions of test data X: (2957, 20000) Y: (2957, 71)


# fourgram of plot

In [0]:
vectorizer =TfidfVectorizer(min_df=5,ngram_range=(4,4),max_features=5000)
x_train_multilabel2 = vectorizer.fit_transform(x_train1)
x_cv_multilabel2 = vectorizer.transform(x_cv1)
x_test_multilabel2 = vectorizer.transform(x_test1)

In [0]:
print("Dimensions of train data X:",x_train_multilabel2.shape, "Y :",y_train.shape)
print("Dimensions of cv data X:",x_cv_multilabel2.shape, "Y :",y_cv.shape)
print("Dimensions of test data X:",x_test_multilabel2.shape,"Y:",y_test.shape)

Dimensions of train data X: (9436, 3590) Y : (9436, 71)
Dimensions of cv data X: (2359, 3590) Y : (2359, 71)
Dimensions of test data X: (2957, 3590) Y: (2957, 71)


# unigram and bigram of title

In [0]:
vectorizer =TfidfVectorizer(ngram_range=(1,2),max_features=10000)
x_train_multilabel3 = vectorizer.fit_transform(x_train2)
x_cv_multilabel3 = vectorizer.transform(x_cv2)
x_test_multilabel3 = vectorizer.transform(x_test2)

In [0]:
print("Dimensions of train data X:",x_train_multilabel3.shape, "Y :",y_train.shape)
print("Dimensions of cv data X:",x_cv_multilabel3.shape, "Y :",y_cv.shape)
print("Dimensions of test data X:",x_test_multilabel3.shape,"Y:",y_test.shape)

Dimensions of train data X: (9436, 10000) Y : (9436, 71)
Dimensions of cv data X: (2359, 10000) Y : (2359, 71)
Dimensions of test data X: (2957, 10000) Y: (2957, 71)


# char gram of plot

In [0]:
vectorizer =TfidfVectorizer(min_df=10,analyzer='char',ngram_range=(4,6),max_features=20000)
x_train_multilabel4 = vectorizer.fit_transform(x_train1)
x_cv_multilabel4 = vectorizer.transform(x_cv1)
x_test_multilabel4 = vectorizer.transform(x_test1)

# matrix factorization using nmf

In [0]:
vectorizer =CountVectorizer(min_df=10)
x_train_multilabl6 = vectorizer.fit_transform(x_train1)
x_cv_multilabl6 = vectorizer.transform(x_cv1)
x_test_multilabl6 = vectorizer.transform(x_test1)

In [0]:
print("Dimensions of train data X:",x_train_multilabl6.shape, "Y :",y_train.shape)
print("Dimensions of cv data X:",x_cv_multilabl6.shape, "Y :",y_cv.shape)
print("Dimensions of test data X:",x_test_multilabl6.shape,"Y:",y_test.shape)

Dimensions of train data X: (9436, 13780) Y : (9436, 71)
Dimensions of cv data X: (2359, 13780) Y : (2359, 71)
Dimensions of test data X: (2957, 13780) Y: (2957, 71)


In [0]:
from sklearn.decomposition import NMF
nmf=NMF(n_components=200)
tr1=nmf.fit_transform(x_train_multilabl6)
cv1=nmf.transform(x_cv_multilabl6)
test1=nmf.transform(x_test_multilabl6)

## (unigram+bigram+trigram+fourgram+word2vec) of plot+unigram and bigram of title+nmf of plot

In [0]:
from scipy.sparse import hstack
x_train_multilabl = hstack([x_train_multilabel1,x_train_multilabel2,x_train_multilabel3,x_train_multilabel4,tr,tr1])
print(x_train_multilabl.shape)
x_cv_multilabl = hstack([x_cv_multilabel1,x_cv_multilabel2,x_cv_multilabel3,x_cv_multilabel4,cv,cv1])
print(x_cv_multilabl.shape)

x_test_multilabl = hstack([x_test_multilabel1,x_test_multilabel2,x_test_multilabel3,x_test_multilabel4,test,test1])
print(x_test_multilabl.shape)

(9436, 54000)
(2359, 54000)
(2957, 54000)


# logistic regression

In [0]:
start = datetime.now()
alpha=[0.01,0.1,1,10]
for j in alpha:
    classifier_1 = OneVsRestClassifier(LogisticRegression(C=j,penalty='l1',tol=0.001))
    classifier_1.fit(x_train_multilabl, y_train)
    y_pred_prob = classifier_1.predict_proba(x_train_multilabl)
    i=0.2
    predictions_1 = (y_pred_prob >= i).astype(int)
    y_pred_probc = classifier_1.predict_proba(x_cv_multilabl)
 
    predictions_2 = (y_pred_probc >= i).astype(int)
    
    f1 = f1_score(y_train, predictions_1, average='micro')
    f2 = f1_score(y_cv, predictions_2, average='micro')
    print("Micro-average quality numbers for C=",j)
    print(" F1-measure for train: {:.4f}".format( f1))
    print(" F1-measure for cv: {:.4f}".format( f2))
print("Time taken to run this cell :", datetime.now() - start)

Micro-average quality numbers for C= 0.01
 F1-measure for train: 0.3378
 F1-measure for cv: 0.3405
Micro-average quality numbers for C= 0.1
 F1-measure for train: 0.3867
 F1-measure for cv: 0.3786
Micro-average quality numbers for C= 1
 F1-measure for train: 0.5037
 F1-measure for cv: 0.4015
Micro-average quality numbers for C= 10
 F1-measure for train: 0.9620
 F1-measure for cv: 0.3400
Time taken to run this cell : 0:38:36.247697


# decreasing threshold on test

In [0]:
classifier_1 = OneVsRestClassifier(LogisticRegression(C=1,penalty='l2',tol=0.001))
classifier_1.fit(x_train_multilabl, y_train)
y_pred_prob = classifier_1.predict_proba(x_test_multilabl)
i=0.2
y_pred_new = (y_pred_prob >= i).astype(int)
f2 = f1_score(y_test, y_pred_new, average='micro')
print("Micro-average quality numbers for C=0.01 and threshold=",i)
print(" F1-measure for test: {:.4f}".format( f2))

Micro-average quality numbers for C=0.01 and threshold= 0.2
 F1-measure for test: 0.3979


# naive bayes

In [0]:
x_train_multilabl = hstack([x_train_multilabel1,x_train_multilabel2,x_train_multilabel3,tr,tr1,tr2])
print(x_train_multilabl.shape)
x_cv_multilabl = hstack([x_cv_multilabel1,x_cv_multilabel2,x_cv_multilabel3,cv,cv1,cv2])
print(x_cv_multilabl.shape)

x_test_multilabl = hstack([x_test_multilabel1,x_test_multilabel2,x_test_multilabel3,test,test1,test2])
print(x_test_multilabl.shape)

(9436, 34000)
(2359, 34000)
(2957, 34000)


In [0]:
from sklearn.naive_bayes import BernoulliNB
start = datetime.now()
alpha=[0.001,0.01,0.1,1,10]
for i in alpha:
    classifier_1 = OneVsRestClassifier(BernoulliNB(alpha=i))
    classifier_1.fit(x_train_multilabl, y_train)
    predictions_1 = classifier_1.predict(x_train_multilabl)
    predictions_2 = classifier_1.predict(x_cv_multilabl)
    f1 = f1_score(y_train, predictions_1, average='micro')
    f2 = f1_score(y_cv, predictions_2, average='micro')
    print("Micro-average quality numbers for C=",i)
    print(" F1-measure for train: {:.4f}".format( f1))
    print(" F1-measure for cv: {:.4f}".format( f2))
print("Time taken to run this cell :", datetime.now() - start)

Micro-average quality numbers for C= 0.001
 F1-measure for train: 0.6216
 F1-measure for cv: 0.3447
Micro-average quality numbers for C= 0.01
 F1-measure for train: 0.5674
 F1-measure for cv: 0.3393
Micro-average quality numbers for C= 0.1
 F1-measure for train: 0.4540
 F1-measure for cv: 0.3024
Micro-average quality numbers for C= 1
 F1-measure for train: 0.3152
 F1-measure for cv: 0.2559
Micro-average quality numbers for C= 10
 F1-measure for train: 0.2087
 F1-measure for cv: 0.1847
Time taken to run this cell : 0:04:25.800332


In [0]:
classifier_1 = OneVsRestClassifier(BernoulliNB(alpha=0.001))
classifier_1.fit(x_train_multilabl, y_train)
y_pred_prob = classifier_1.predict_proba(x_test_multilabl)
y_pred_new = (y_pred_prob >= 0.01).astype(int)
f2 = f1_score(y_test, y_pred_new, average='micro')
print("Micro-average quality numbers for C=0.001 ")
print(" F1-measure for test: {:.4f}".format( f2))

Micro-average quality numbers for C=0.001 
 F1-measure for test: 0.3454


# using deep learning technique

In [0]:
count_vect = CountVectorizer(min_df=10) #in scikit-learn CountVectorizer(min_df=10, max_features=500) 
X=count_vect.fit_transform(x_train1)
top_feat=count_vect.get_feature_names()

In [0]:
X.shape

(9436, 13780)

In [0]:
#placing  words in reviews in a list
word=[]
for sent in x_train1:
    wrds = sent.split()
    for wrd in wrds:
        word.append(wrd)
    

In [0]:
a=np.arange(13780)
feat10k=dict( zip( top_feat, a))

In [0]:
feat=np.zeros(13780,dtype='int')
for wrd in word:
  if feat10k.get(wrd,-1)>0:
    feat[feat10k[wrd]]+=1
     

In [0]:
sort_feat=np.argsort(feat)
sorted_list=[]
for i in sort_feat:
    sorted_list.append(top_feat[i])
    
    

In [0]:
sort_freq=dict( zip( sorted_list, sort_feat))

In [0]:
ranktr=[]
for sent in x_train1:
    lst=[]
    wrds = sent.split()
    for wrd in wrds:
        if feat10k.get(wrd,-1)>0:
            lst.append(sort_freq[wrd])
    ranktr.append(lst)   

In [0]:
rankcv=[]
for sent in x_cv1:
    lst=[]
    wrds = sent.split()
    for wrd in wrds:
        if feat10k.get(wrd,-1)>0:
            lst.append(sort_freq[wrd])
    rankcv.append(lst)   

In [0]:
ranktest=[]
for sent in x_test1:
    lst=[]
    wrds = sent.split()
    for wrd in wrds:
        if feat10k.get(wrd,-1)>0:
            lst.append(sort_freq[wrd])
    ranktest.append(lst)   

In [0]:
a=max(x_train1, key=len).strip()

In [0]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Bidirectional
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
# fix random seed for reproducibility
np.random.seed(7)


Using TensorFlow backend.


In [0]:
max_review_length = 1000
X_train = sequence.pad_sequences(ranktr, maxlen=max_review_length)
X_cv   = sequence.pad_sequences(rankcv, maxlen=max_review_length)
X_test = sequence.pad_sequences(ranktest, maxlen=max_review_length)

print(X_train.shape)
print(X_train[1])

(9436, 1000)
[    0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0  

In [0]:
embedding_vecor_length = 300
model_1 = Sequential()
model_1.add(Embedding(13780+1, embedding_vecor_length, input_length=max_review_length))
model_1.add(LSTM(128))
model_1.add(Dense(71, activation='sigmoid'))
model_1.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


W0817 10:11:53.313753 140710187665280 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0817 10:11:53.354623 140710187665280 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0817 10:11:53.361517 140710187665280 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0817 10:11:53.683286 140710187665280 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0817 10:11:53.710934 140710187665280 deprecation_wrappe

In [0]:
model_1.fit(X_train, y_train, epochs=5, batch_size=512,validation_data=(X_cv,y_cv))

W0817 10:11:54.956021 140710187665280 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:986: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.



Train on 9436 samples, validate on 2359 samples
Epoch 1/5
9436/9436 [==============================] - 51s 5ms/step - loss: 0.5485 - acc: 0.8645 - val_loss: 0.2306 - val_acc: 0.9574
Epoch 2/5
9436/9436 [==============================] - 47s 5ms/step - loss: 0.1645 - acc: 0.9583 - val_loss: 0.1449 - val_acc: 0.9574
Epoch 3/5
9436/9436 [==============================] - 46s 5ms/step - loss: 0.1411 - acc: 0.9583 - val_loss: 0.1421 - val_acc: 0.9574
Epoch 4/5
9436/9436 [==============================] - 46s 5ms/step - loss: 0.1393 - acc: 0.9583 - val_loss: 0.1412 - val_acc: 0.9574
Epoch 5/5
9436/9436 [==============================] - 46s 5ms/step - loss: 0.1387 - acc: 0.9583 - val_loss: 0.1410 - val_acc: 0.9574


In [0]:
predict_probtr = model_1.predict(X_train)


In [0]:
t=[0.5,0.2,0.1,0.05]#threshold

In [0]:
for i in t:
  y_predtr = (predict_probtr >= i).astype(int)
  f1 = f1_score(y_train, y_predtr, average='micro')
  print(" F1-measure for train: {:.4f} and threshold={}".format( f1,i))

 F1-measure for train: 0.0000 and threshold=0.5
 F1-measure for train: 0.2773 and threshold=0.2
 F1-measure for train: 0.3053 and threshold=0.1
 F1-measure for train: 0.2485 and threshold=0.05


# taking threshold as 0.1

In [0]:
predict_prob = model_1.predict(X_test)


In [0]:
predict_prob[0]

array([0.01858133, 0.04362813, 0.00821382, 0.00974792, 0.00627637,
       0.01454931, 0.00741696, 0.02862522, 0.00422007, 0.01805571,
       0.00680026, 0.01417279, 0.0356462 , 0.00719705, 0.0037584 ,
       0.00538141, 0.00522521, 0.12843624, 0.00783843, 0.02855575,
       0.17216185, 0.01242733, 0.02835459, 0.01230884, 0.02762339,
       0.04805008, 0.03844845, 0.00480804, 0.19711196, 0.06184345,
       0.03073347, 0.00541219, 0.00966036, 0.01791635, 0.01042733,
       0.00973195, 0.03316283, 0.05565515, 0.04421234, 0.00669426,
       0.01055536, 0.00592789, 0.02996501, 0.37207168, 0.03519765,
       0.05066758, 0.00272527, 0.03214604, 0.01680574, 0.0126853 ,
       0.0106259 , 0.01619208, 0.13411337, 0.01968622, 0.00567257,
       0.01547539, 0.16526577, 0.18440023, 0.04246202, 0.05527481,
       0.02168334, 0.01704258, 0.02548701, 0.01247486, 0.00274321,
       0.07060099, 0.00799596, 0.03832561, 0.28975213, 0.00473303,
       0.00628313], dtype=float32)

In [0]:
y_pred_new = (predict_prob >= 0.1).astype(int)
f2 = f1_score(y_test, y_pred_new, average='micro')
print(" F1-measure for test: {:.4f}".format( f2))

 F1-measure for test: 0.3100


In [0]:
print(y_test)

  (0, 56)	1
  (0, 65)	1
  (1, 52)	1
  (2, 29)	1
  (2, 68)	1
  (2, 26)	1
  (2, 12)	1
  (2, 57)	1
  (3, 20)	1
  (3, 48)	1
  (4, 56)	1
  (4, 65)	1
  (4, 68)	1
  (4, 43)	1
  (4, 28)	1
  (5, 68)	1
  (5, 12)	1
  (5, 43)	1
  (5, 28)	1
  (5, 24)	1
  (5, 15)	1
  (5, 61)	1
  (6, 52)	1
  (6, 68)	1
  (6, 43)	1
  :	:
  (2940, 43)	1
  (2941, 65)	1
  (2942, 43)	1
  (2943, 47)	1
  (2944, 56)	1
  (2944, 65)	1
  (2944, 68)	1
  (2944, 45)	1
  (2945, 43)	1
  (2945, 38)	1
  (2946, 7)	1
  (2947, 28)	1
  (2947, 17)	1
  (2947, 47)	1
  (2947, 37)	1
  (2948, 12)	1
  (2949, 56)	1
  (2949, 57)	1
  (2950, 47)	1
  (2951, 43)	1
  (2952, 43)	1
  (2953, 68)	1
  (2954, 43)	1
  (2955, 43)	1
  (2956, 43)	1


# checking the predicted values

In [0]:
y_pred_new[2][57]

1

In [0]:
y_pred_new[2][68]

1

In [0]:
y_pred_new[2]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0])

## out of 4 tags in actual dataset at y_test[2] this model is predicting 2 correctly

## the best metrics will be to check recall value.i.e how many times we are actually predicting correct label

In [0]:
f2 = recall_score(y_test, y_pred_new, average='micro')

In [0]:
print(" recall-measure for test: {:.4f}".format( f2))

 recall-measure for test: 0.5620


# let check precision also

In [0]:
f2 = precision_score(y_test, y_pred_new, average='micro')

In [0]:
print(" precision-measure for test: {:.4f}".format( f2))

 precision-measure for test: 0.2140


### the recall value is great.approx half of the actual value we are able to recall 

# TOPIC MODELLING

## for using randomised search cv and 3 fold cv we merge train and cv together

In [0]:
x_train1=preprocess_plot[2957:]
x_test1=preprocess_plot[:2957]

y_train = multilabel_y[2957:]
y_test = multilabel_y[:2957]


In [0]:
vectorizer =TfidfVectorizer(min_df=10,ngram_range=(1,3),max_features=20000)
x_train_multilabel1 = vectorizer.fit_transform(x_train1)
x_test_multilabel1 = vectorizer.transform(x_test1)

In [28]:
x_train_multilabel1.shape

(11795, 20000)

In [0]:
from sklearn.decomposition import LatentDirichletAllocation
lda=LatentDirichletAllocation()
tr2=lda.fit_transform(x_train_multilabel1)
test2=lda.transform(x_test_multilabel1)

In [29]:
from scipy.sparse import hstack
x_train_multilabl = hstack([x_train_multilabel1,tr2])
print(x_train_multilabl.shape)
x_test_multilabl = hstack([x_test_multilabel1,test2])
print(x_test_multilabl.shape)

(11795, 20010)
(2957, 20010)


# logistic regression

In [0]:
# I am keeping same hyperparameter which i got after tuning above
classifier_1 = OneVsRestClassifier(LogisticRegression(C=1,penalty='l2',tol=0.001))
classifier_1.fit(x_train_multilabl, y_train)
yprobtr= classifier_1.predict_proba(x_train_multilabl)
y_pred_prob = classifier_1.predict_proba(x_test_multilabl )
i=0.2 
y_predtr = (yprobtr >= i).astype(int)
y_pred_new = (y_pred_prob >= i).astype(int)
f1 = f1_score(y_train,y_predtr, average='micro')
f2 = f1_score(y_test, y_pred_new, average='micro')
print("Micro-average quality numbers for C=1 and threshold=",i)
print(" F1-measure for test: {:.4f}".format( f1))
print(" F1-measure for test: {:.4f}".format( f2))

Micro-average quality numbers for C=1 and threshold= 0.2
 F1-measure for test: 0.5466
 F1-measure for test: 0.4020


# LightGBM

In [0]:
import lightgbm as lgb
from sklearn.model_selection import RandomizedSearchCV
dtr =  OneVsRestClassifier(lgb.LGBMClassifier())
prams={
    
     'estimator__num_iterations':[100,200,500,1000,2000],
     'estimator__max_depth':[3,5,10,30]
}
random_cfl=RandomizedSearchCV(dtr,param_distributions=prams,scoring='f1_micro',verbose=10,n_jobs=-1)
random_cfl.fit(x_train_multilabel1,y_train)

Fitting 3 folds for each of 10 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:  8.1min
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed: 29.6min
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed: 92.5min
[Parallel(n_jobs=-1)]: Done  14 tasks      | elapsed: 117.0min
[Parallel(n_jobs=-1)]: Done  21 tasks      | elapsed: 168.6min
[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed: 215.3min finished


RandomizedSearchCV(cv='warn', error_score='raise-deprecating',
                   estimator=OneVsRestClassifier(estimator=LGBMClassifier(boosting_type='gbdt',
                                                                          class_weight=None,
                                                                          colsample_bytree=1.0,
                                                                          importance_type='split',
                                                                          learning_rate=0.1,
                                                                          max_depth=-1,
                                                                          min_child_samples=20,
                                                                          min_child_weight=0.001,
                                                                          min_split_gain=0.0,
                                                                          n_estimato

In [0]:
random_cfl.best_params_

{'estimator__max_depth': 3, 'estimator__num_iterations': 500}

In [30]:
import lightgbm as lgb
dtr =  OneVsRestClassifier(lgb.LGBMClassifier(max_depth=3,num_iterations=500))
dtr.fit(x_train_multilabl, y_train)


OneVsRestClassifier(estimator=LGBMClassifier(boosting_type='gbdt',
                                             class_weight=None,
                                             colsample_bytree=1.0,
                                             importance_type='split',
                                             learning_rate=0.1, max_depth=3,
                                             min_child_samples=20,
                                             min_child_weight=0.001,
                                             min_split_gain=0.0,
                                             n_estimators=100, n_jobs=-1,
                                             num_iterations=500, num_leaves=31,
                                             objective=None, random_state=None,
                                             reg_alpha=0.0, reg_lambda=0.0,
                                             silent=True, subsample=1.0,
                                             subsample_for_bin=200000,


In [0]:
ytrprob=dtr.predict_proba(x_train_multilabl)

In [0]:
ytestprob=dtr.predict_proba(x_test_multilabl)

In [33]:
threshold=[0.5,0.3,0.2,0.1]
for i in threshold:
  y_predtr = (ytrprob >= i).astype(int)
  y_predtest = (ytestprob >= i).astype(int)
  f1 = f1_score(y_train,y_predtr, average='micro')
  f2 = f1_score(y_test, y_predtest, average='micro')
  print("Micro-average quality numbers for C=1 and threshold=",i)
  print(" F1-measure for test: {:.4f}".format( f1))
  print(" F1-measure for test: {:.4f}".format( f2))

Micro-average quality numbers for C=1 and threshold= 0.5
 F1-measure for test: 0.8106
 F1-measure for test: 0.2981
Micro-average quality numbers for C=1 and threshold= 0.3
 F1-measure for test: 0.8812
 F1-measure for test: 0.3861
Micro-average quality numbers for C=1 and threshold= 0.2
 F1-measure for test: 0.8380
 F1-measure for test: 0.4087
Micro-average quality numbers for C=1 and threshold= 0.1
 F1-measure for test: 0.6569
 F1-measure for test: 0.3902


In [1]:
from prettytable import PrettyTable

# Initializing table object
x = PrettyTable()

x.field_names = ["vectorizer","Model","train f1","test f1"]
x.add_row(["tfidf","logistic regression","0.4015","0.3979" ])
x.add_row(["tfidf","naive bayes","0.3447","0.3454" ])
x.add_row(["deep learning","LSTM","0.3173","0.3215" ])
x.add_row(["topic medelling+ tfidf","Logistic Regression","0.54","0.40" ])
x.add_row(["topic medelling+ tfidf","LightGBM","0.83","0.408" ])
print(x)

+------------------------+---------------------+----------+---------+
|       vectorizer       |        Model        | train f1 | test f1 |
+------------------------+---------------------+----------+---------+
|         tfidf          | logistic regression |  0.4015  |  0.3979 |
|         tfidf          |     naive bayes     |  0.3447  |  0.3454 |
|     deep learning      |         LSTM        |  0.3173  |  0.3215 |
| topic medelling+ tfidf | Logistic Regression |   0.54   |   0.40  |
| topic medelling+ tfidf |       LightGBM      |   0.83   |  0.408  |
+------------------------+---------------------+----------+---------+


# conclusion

reference: https://www.kaggle.com/cryptexcode/mpst-movie-plot-synopses-with-tags

1. I got a significant improvement of around 5% from original research paper mentioned in above link.

2. Experimenting with lots of featurization and including topic modelling  improved my score.

3. Hacks using threshold was very effecient and improved score.

4. tags need to be preprocessed as it contained multiple duplicate tags with a space in front.after preprocessing i got 71 tags outof 141

5. I used every possible approach i.e tfidf,matrix factorization techniques and deep learning.

6. The final best micro f1 on test was .408.

7. The given dataset was very small and plot featurization increased dimention .
   due to lack of sufficient data point deep learning didnt perform well.

8. I trained model with train and test split specified by the dataset.

9. I cared about data leakage.

10. Decision trees was the worst performer because of high dimention.
